# Driver distraction classification using Convolutional Neural Networks

### Import libraries

In [20]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### Data preprocessing

In [2]:
workingdir = os.path.abspath('')
print(workingdir)
csvdata = pd.read_csv(os.path.join(workingdir + '/state-farm-distracted-driver-detection//driver_imgs_list.csv'))
csvdata.head(2)

C:\Users\thami\Documents\GitHub\MachineLearningDriverdistraction


,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg


In [3]:
#Upload Training Data
trainingdirectory = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/')

### Image resize

In [4]:
def resizeimage(path, img_rows, img_cols, color_type=3):
    img = cv2.imread(path,0)
    resizedimage = cv2.resize(img, (img_cols, img_rows))
    return resizedimage

### Training and Validation data split

In [67]:
image = csvdata[['img']]
classname = csvdata[['classname']]
imagetrain, imagevalidation, classnametrain, classnamevalidation = train_test_split(image, classname, test_size=0.20)

In [6]:
trainingimage = []
for value in range(len(imagetrain)):
    path = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/{}/{}').format(classnametrain.iloc[value,0], 
                                                                                                    imagetrain.iloc[value,0])
    resizedimage = resizeimage(path,32,32)
    trainingimage.append(resizedimage)

In [7]:
validationimage = []
for value in range(len(imagevalidation)):
    path = os.path.join(workingdir + '/state-farm-distracted-driver-detection/imgs/train/{}/{}').format(classnamevalidation.iloc[value,0],
                                                                                          imagevalidation.iloc[value,0])
    resizedimage = resizeimage(path,32,32)
    validationimage.append(resizedimage)

In [35]:
trainingimagearray = np.array(trainingimage)
validationimagearray = np.array(validationimage)
trainingimagearray, validationimagearray = trainingimagearray/255.0, validationimagearray/255.0

In [54]:
trainingimagearray = trainingimagearray.reshape(17939,32,32,1)
validationimagearray = validationimagearray.reshape(4485,32,32,1)

In [68]:
labelencoder = LabelEncoder()
classnametrain['encodercolumn'] = labelencoder.fit_transform(classnametrain)
classnamevalidation['encodercolumn'] = labelencoder.fit_transform(classnamevalidation)

### Model

In [58]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 4, 4, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_8 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 64)              

In [69]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(trainingimagearray, classnametrain['encodercolumn'], 
                    epochs=25,validation_data=(validationimagearray, classnamevalidation['encodercolumn']))
model.save("cnnmodel.h5")

Epoch 1/25
561/561 [==============================] - 11s 19ms/step - loss: 2.2998 - accuracy: 0.1123 - val_loss: 2.3014 - val_accuracy: 0.1059
Epoch 2/25
561/561 [==============================] - 13s 23ms/step - loss: 2.2997 - accuracy: 0.1123 - val_loss: 2.3012 - val_accuracy: 0.1059
Epoch 3/25
561/561 [==============================] - 10s 19ms/step - loss: 2.2998 - accuracy: 0.1123 - val_loss: 2.3014 - val_accuracy: 0.1059
Epoch 4/25
561/561 [==============================] - 5s 8ms/step - loss: 2.2997 - accuracy: 0.1123 - val_loss: 2.3017 - val_accuracy: 0.1059
Epoch 5/25
561/561 [==============================] - 12s 21ms/step - loss: 2.2998 - accuracy: 0.1123 - val_loss: 2.3016 - val_accuracy: 0.1059
Epoch 6/25
561/561 [==============================] - 13s 22ms/step - loss: 2.2997 - accuracy: 0.1123 - val_loss: 2.3015 - val_accuracy: 0.1059
Epoch 7/25
561/561 [==============================] - 11s 19ms/step - loss: 2.2998 - accuracy: 0.1123 - val_loss: 2.3015 - val_accuracy: 0

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1.1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(validationimagearray, classnamevalidation['encodercolumn'], verbose=2)